In [2]:
#!/usr/bin/env python3
"""
Patient Timeline Creator
Creates a comprehensive timeline for patients by combining data from multiple tables.
"""

import pandas as pd
import duckdb as db
import os
from datetime import datetime
from typing import Dict, List, Tuple, Optional

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:

def get_database_connection():
    """Create and return a connection to the clinisys_all database"""
    path_to_db = '../../database/huntington_data_lake.duckdb'
    conn = db.connect(path_to_db, read_only=True)
    
    print(f"Connected to database: {path_to_db}")
    print(f"Database file exists: {os.path.exists(path_to_db)}")
    
    return conn

def extract_timeline_data_for_patient(conn, test_prontuario):
    """Extract timeline data for a specific patient"""
    query = f"""
        SELECT * FROM gold.all_patients_timeline
        WHERE prontuario = {test_prontuario}
    """
    return conn.execute(query).fetchdf()

def get_finops_data(conn, test_prontuario):
    """Get finops data for a specific patient"""
    query = f"""
        SELECT * FROM gold.finops_summary
        WHERE prontuario = {test_prontuario}
    """
    return conn.execute(query).fetchdf()

def get_mescladas(conn, test_prontuario):
    """Get mescladas data for a specific patient"""
    query = f"""
        SELECT * FROM silver.mesclada_vendas
        WHERE prontuario = {test_prontuario}
    """
    return conn.execute(query).fetchdf()

def main(test_prontuario):
    """Main function to create patient timeline"""
    
    # Test with a single patient first
    # test_prontuario = 175583
    
    print(f"Creating timeline for patient: {test_prontuario}")
    
    # Get database connection
    conn = get_database_connection()
    
    try:
        # Extract timeline data
        timeline_df = extract_timeline_data_for_patient(conn, test_prontuario)
       
        finops_df = get_finops_data(conn, test_prontuario)

        mescladas_df = get_mescladas(conn, test_prontuario)
        return timeline_df, finops_df, mescladas_df
        
    finally:
        conn.close()

In [16]:
if __name__ == "__main__":

    ## casos para testes iniciais

    # patient_id = 825_890 # Claudia
    # patient_id = 175_583 # Renata
    # patient_id = 220_783 # caso cabelud0:: - 26 tentativas
    # patient_id = 182_925 # caso completo
    # patient_id = 162_173 # caso bonito: 5 ciclos - 5 pagamentos

    ## resolvidos
    # patient_id = 641_950 # 11 cycles without payment
    # patient_id = 150_491 # ovo recepção
    # patient_id = 155_282 # medico externo
    # patient_id = 738_349 # (marido 738_350 não aparece no clinisys) -- sem pagamento
    # patient_id = 839574 # utero de substituicao (815299)
    # patient_id = 842663 # doadora
    # patient_id = 678_207 # numero cliente duplicado no diario_vendas (78207) - BH
    # patient_id = 831_609 # pagamento no ID errado do paciente, mas certo no marido (CLiente = 833_788)
    # patient_id = 662_298 # paciente 50347 em mescladas, cliente 62298 no diario - BH
    
    # patient_id = 162733 # Beshaatova - CLIENTE/PACIENTE 64645 - Filipe
    # patient_id = 832_556 # utero de substituição - pago por 831_609 - Filipe
    # patient_id = 881_426 # utero de substituição - 126881 - Filipe

    ## abertos

    # patient_id = 880_484 # erro de migração de cadastro - pago  cliente 55623
    # patient_id = 867_007 # pago - nomes diferentes no clinisys e mescladas (erro de cadastro no Totvs)
    # patient_id = 875_610 # autorizado pela gerencia, sem custo
    # patient_id = 178125 # pago em 2021, primeira transferencia em 2025

    ## sem pagamento ou sem faturamento
    # patient_id = 166_697 # FOT R$ 4695 em 11/06/2025 + biópisia
    # patient_id = 172_179 # funcionaria 
    # patient_id = 885_517  # Matheus Roque - pago na Mater Prime
    # patient_id = 879_641 # Matheus Roque - pago na Mater Prime
    # patient_id = 887_035 # relato que está pago no orçamento, mas não lançado e não achamos NF
    # patient_id = 873_039 # não pago - FET
    

    ## depois da mudança no código
    patient_id = 655_506
    patient_id = 895057 



    timeline_df, finops_df, mescladas_df = main(patient_id)


Creating timeline for patient: 895057
Connected to database: ../../database/huntington_data_lake.duckdb
Database file exists: True


In [17]:
finops_df

,prontuario,cycle_with_transfer,cycle_without_transfer,cycle_total,treatment_paid_count,treatment_paid_total,timeline_first_date,timeline_last_date,billing_first_date,billing_last_date,prontuario_genitores,medico_nome,flag_is_donor,flag_cycles_no_payments,flag_more_cycles_than_payments,flag_no_cycles_but_payments,flag_less_cycles_than_payments,timeline_unidade
0,895057.0,0,0,0,1.0,29338.2,NaT,NaT,2025-09-05,2025-09-05,895057.0,Fernanda de Paula Rodrigues,0,0,0,1,1,1. HTT SP - Ibirapuera


In [18]:
timeline_df

,prontuario,event_id,event_date,reference,reference_value,tentativa,unidade,resultado_tratamento,flag_date_estimated,additional_info
0,895057.0,37677,2025-09-24,tratamentos,Ciclo a Fresco FIV,1,1. HTT SP - Ibirapuera,None,True,{}
1,895057.0,1497687,2025-09-22,extrato_atendimentos,Coleta Seminal: FIV,None,1. HTT SP - Ibirapuera,None,False,{}
2,895057.0,1497530,2025-09-22,extrato_atendimentos,Aspiração Folicular: FIV,None,1. HTT SP - Ibirapuera,None,False,{}
3,895057.0,1496321,2025-09-19,extrato_atendimentos,4º US Ciclo,None,1. HTT SP - Ibirapuera,None,False,{}
4,895057.0,1494467,2025-09-17,extrato_atendimentos,3º US Ciclo,None,1. HTT SP - Ibirapuera,None,False,{}
5,895057.0,1490975,2025-09-15,extrato_atendimentos,2º US Ciclo,None,1. HTT SP - Ibirapuera,None,False,{}
6,895057.0,1478703,2025-09-09,extrato_atendimentos,1° US Ciclo,None,1. HTT SP - Ibirapuera,None,False,{}
7,895057.0,1487557,2025-09-08,extrato_atendimentos,1° US Ciclo,None,1. HTT SP - Ibirapuera,None,False,{}
8,895057.0,1470140,2025-08-19,extrato_atendimentos,1ª Consulta de Reprodução Humana,None,1. HTT SP - Ibirapuera,None,False,{}


In [19]:
mescladas_df.sort_values(by='DT Emissao', ascending=False)

,Cliente,Nome,Paciente,Nom Paciente,prontuario,DT Emissao,Descricao,Qntd.,Total,Descrição Gerencial,Loja,Tipo da nota,Numero,Serie Docto.,NF Eletr.,Vend. 1,Médico,Cliente_totvs,Operador,Produto,Valor Mercadoria,Custo,Custo Unit,Desconto,Unidade,Mês,Ano,Cta-Ctbl,Cta-Ctbl Eugin,Interno/Externo,Descrição Mapping Actividad,Ciclos,Qnt Cons.,Grp,Descr.TES,Lead Time,Data do Ciclo,Fez Ciclo?,line_number,extraction_timestamp,file_name
1,895057,MARIA CLARA MU,895057,MARIA CLARA MURAD BARACAT FERREIRA,895057,2025-09-08,ESTRADIOL (40316246),1.0,160.0,Análises Clínicas,10150,502,31034,RPS,30917,12,FERNANDA DE PAULA RODRIGUES,899222,,S3564.42,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100018,,,Análises Clínicas,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,2640,20250916_153309,Mescladas2.xlsx
2,895057,MARIA CLARA MU,895057,MARIA CLARA MURAD BARACAT FERREIRA,895057,2025-09-08,FSH - HORMONIO FOLICULO-ESTIMULANTE (403,1.0,160.0,Análises Clínicas,10150,502,31034,RPS,30917,12,FERNANDA DE PAULA RODRIGUES,899222,,S3564.45,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100018,,,Análises Clínicas,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,2641,20250916_153309,Mescladas2.xlsx
3,895057,MARIA CLARA MU,895057,MARIA CLARA MURAD BARACAT FERREIRA,895057,2025-09-08,PROGESTERONA (40316408),1.0,160.0,Análises Clínicas,10150,502,31034,RPS,30917,12,FERNANDA DE PAULA RODRIGUES,899222,,S3564.55,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100018,,,Análises Clínicas,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,2642,20250916_153309,Mescladas2.xlsx
0,895057,MARIA CLARA MU,895057,MARIA CLARA MURAD BARACAT FERREIRA,895057,2025-09-05,FIV - MEDICOS INTERNOS,1.0,29338.2,FIV,10150,502,30627,RPS,30503,12,FERNANDA DE PAULA RODRIGUES,899222,,S3551,0.0,0.0,0.0,1235.8,Ibirapuera,0,0,31110100001,,,FIV,1,0,1,PRESTACAO DE SERVICO,None,NaT,False,2209,20250916_153309,Mescladas2.xlsx
4,895057,MARIA CLARA MU,895057,MARIA CLARA MURAD BARACAT FERREIRA,895057,2025-08-20,PRIMEIRA CONSULTA REPRODUCAO HUMANA,1.0,990.0,Consultas de CRH,10150,502,27788,RPS,27637,12,FERNANDA DE PAULA RODRIGUES,899222,,S3555.01,0.0,0.0,0.0,110.0,Ibirapuera,0,0,31110100025,,,1ª Consulta,0,0,1,PRESTACAO DE SERVICO,16,2025-09-05,True,395162,20250916_153309,Mescladas2.xlsx
